In [1]:
import pandas as pd
import plotly.express as px
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import seaborn as sns

from mlxtend.frequent_patterns import apriori, association_rules

## EDA

In [2]:
df_games = pd.read_csv('./game_sales.csv',sep=",")

In [3]:
df_games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Client ID  10000 non-null  object
 1   Game ID    10000 non-null  object
 2   Game Name  10000 non-null  object
dtypes: object(3)
memory usage: 234.5+ KB


In [4]:
df_games.rename(columns={"Client ID":"Client_ID",
                         "Game ID":"Game_ID",
                         "Game Name": "Game_Name"}, inplace=True)

In [5]:
df_games.head()

,Client_ID,Game_ID,Game_Name
0,CLIENT-0369,GAME-0013,Elden Ring
1,CLIENT-0287,GAME-0006,Call of Duty: Modern Warfare
2,CLIENT-0108,GAME-0030,PUBG: Battlegrounds
3,CLIENT-0291,GAME-0002,Minecraft
4,CLIENT-0277,GAME-0027,Battlefield 2042


In [6]:
df_games['Game_Name'].nunique()

33

In [7]:
df_games['Game_ID'].nunique()

33

In [8]:
df_games['Client_ID'].nunique()

591

In [9]:
# Quais foram os jogos mais comprados? top 10

jogos_mais_comprados_top10 = df_games.value_counts('Game_Name').head(10)

px.bar(jogos_mais_comprados_top10, color=jogos_mais_comprados_top10.index, orientation='h')

In [10]:
# Porcentagem de ocorrência de cada jogo
jogos_mais_comprados_porcentagem = df_games.value_counts('Game_Name')/len(df_games) * 100

px.bar(jogos_mais_comprados_porcentagem, color=jogos_mais_comprados_porcentagem.index, orientation='h').update_layout(showlegend=False)


## Preparação dos dados

In [11]:
df_games_pivot_table = df_games.pivot_table(index="Client_ID",
                                            columns="Game_Name",
                                            aggfunc="count")

In [12]:
df_games_pivot_table.info()

<class 'pandas.core.frame.DataFrame'>
Index: 591 entries, CLIENT-0001 to CLIENT-0591
Data columns (total 33 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   (Game_ID, ARK: Survival Evolved)         288 non-null    float64
 1   (Game_ID, Among Us)                      304 non-null    float64
 2   (Game_ID, Apex Legends)                  307 non-null    float64
 3   (Game_ID, Assassin's Creed Valhalla)     290 non-null    float64
 4   (Game_ID, Battlefield 2042)              293 non-null    float64
 5   (Game_ID, Call of Duty: Modern Warfare)  331 non-null    float64
 6   (Game_ID, Counter-Strike 2)              320 non-null    float64
 7   (Game_ID, Cyberpunk 2077)                309 non-null    float64
 8   (Game_ID, Destiny 2)                     302 non-null    float64
 9   (Game_ID, Dota 2)                        296 non-null    float64
 10  (Game_ID, Elden Ring)                

In [13]:
df_games_pivot_table.head()

Game_ID                        \
Game_Name   ARK: Survival Evolved Among Us Apex Legends   
Client_ID                                                 
CLIENT-0001                   1.0      1.0          NaN   
CLIENT-0002                   NaN      1.0          1.0   
CLIENT-0003                   1.0      1.0          1.0   
CLIENT-0004                   1.0      NaN          1.0   
CLIENT-0005                   1.0      NaN          1.0   

                                                        \
Game_Name   Assassin's Creed Valhalla Battlefield 2042   
Client_ID                                                
CLIENT-0001                       1.0              NaN   
CLIENT-0002                       1.0              1.0   
CLIENT-0003                       NaN              NaN   
CLIENT-0004                       NaN              1.0   
CLIENT-0005                       NaN              1.0   

                                                                          \
Game_Name   Call of Duty: Modern Warfare Counter-Strike 2 Cyberpunk 2077   
Client_ID                                                                  
CLIENT-0001                          1.0              1.0            1.0   
CLIENT-0002                          1.0              NaN            NaN   
CLIENT-0003                          1.0              1.0            1.0   
CLIENT-0004                          1.0              1.0            NaN   
CLIENT-0005                          1.0              NaN            1.0   

                              ...                                           \
Game_Name   Destiny 2 Dota 2  ... NBA 2K24 Overwatch 2 PUBG: Battlegrounds   
Client_ID                     ...                                            
CLIENT-0001       1.0    NaN  ...      1.0         1.0                 1.0   
CLIENT-0002       NaN    NaN  ...      NaN         1.0                 NaN   
CLIENT-0003       1.0    NaN  ...      1.0         NaN                 1.0   
CLIENT-0004       NaN    1.0  ...      1.0         NaN                 NaN   
CLIENT-0005       1.0    1.0  ...      NaN         1.0                 NaN   

                                                                        \
Game_Name   Red Dead Redemption 2 Resident Evil 4 Roblox Rocket League   
Client_ID                                                                
CLIENT-0001                   1.0             1.0    1.0           1.0   
CLIENT-0002                   NaN             1.0    NaN           1.0   
CLIENT-0003                   NaN             1.0    1.0           1.0   
CLIENT-0004                   1.0             1.0    1.0           1.0   
CLIENT-0005                   NaN             NaN    1.0           1.0   

                                               
Game_Name   The Sims 4 The Witcher 3 Valorant  
Client_ID                                      
CLIENT-0001        1.0           1.0      1.0  
CLIENT-0002        NaN           NaN      1.0  
CLIENT-0003        1.0           NaN      1.0  
CLIENT-0004        NaN           1.0      NaN  
CLIENT-0005        1.0           NaN      NaN  

[5 rows x 33 columns]

In [14]:
# 1. Tratamento de dados: Transforma NaN em 0 e converte tudo para Booleano (True/False)
df_limpo = df_games_pivot_table.fillna(0).astype(bool)

# 2. Ajuste de Hierarquia: Remove o nível superior ("Game_ID") das colunas
# Se o seu DataFrame tiver 2 níveis, isso deixará apenas os nomes dos jogos
df_limpo.columns = df_limpo.columns.droplevel(0)

# 3. Limpeza estética: Remove os nomes residuais dos eixos (Game_ID, Client_ID, etc)
df_limpo.columns.name = None
df_limpo.index.name = None

# Visualizar o resultado final pronto para o Apriori
df_limpo.head()

,ARK: Survival Evolved,Among Us,Apex Legends,Assassin's Creed Valhalla,Battlefield 2042,Call of Duty: Modern Warfare,Counter-Strike 2,Cyberpunk 2077,Destiny 2,Dota 2,...,NBA 2K24,Overwatch 2,PUBG: Battlegrounds,Red Dead Redemption 2,Resident Evil 4,Roblox,Rocket League,The Sims 4,The Witcher 3,Valorant
CLIENT-0001,True,True,False,True,False,True,True,True,True,False,...,True,True,True,True,True,True,True,True,True,True
CLIENT-0002,False,True,True,True,True,True,False,False,False,False,...,False,True,False,False,True,False,True,False,False,True
CLIENT-0003,True,True,True,False,False,True,True,True,True,False,...,True,False,True,False,True,True,True,True,False,True
CLIENT-0004,True,False,True,False,True,True,True,False,False,True,...,True,False,False,True,True,True,True,False,True,False
CLIENT-0005,True,False,True,False,True,True,False,True,True,True,...,False,True,False,False,False,True,True,True,False,False


In [15]:
# Total de clientes únicos (transações)
total_clientes = df_games['Client_ID'].nunique()

# Suporte real de cada jogo
suporte_jogos = (df_games['Game_Name'].value_counts() / total_clientes) * 100

suporte_jogos.head()

Game_Name
Call of Duty: Modern Warfare    56.006768
Counter-Strike 2                54.145516
Grand Theft Auto V              54.145516
Overwatch 2                     53.807107
NBA 2K24                        53.130288
Name: count, dtype: float64

In [16]:
itemsets_frequentes = apriori(df_limpo, min_support=0.02, use_colnames=True, verbose=1, max_len=3)

Processing 16368 combinations | Sampling itemset size 3


In [18]:
# Criar uma coluna para armazenar o tamanho do itemset
itemsets_frequentes['tamanho'] = itemsets_frequentes['itemsets'].apply(lambda x : len(x))

In [20]:
itemsets_frequentes

,support,itemsets,tamanho
0,0.487310,(ARK: Survival Evolved),1
1,0.514382,(Among Us),1
2,0.519459,(Apex Legends),1
3,0.490694,(Assassin's Creed Valhalla),1
4,0.495770,(Battlefield 2042),1
...,...,...,...
6012,0.262267,"(The Witcher 3, Roblox, Valorant)",3
6013,0.247039,"(Rocket League, The Witcher 3, The Sims 4)",3
6014,0.252115,"(Rocket League, The Sims 4, Valorant)",3
6015,0.243655,"(Rocket League, The Witcher 3, Valorant)",3


In [21]:
# Filtrar itemsets com 2 departamentos
itemsets_frequentes[itemsets_frequentes['tamanho'] == 3]

,support,itemsets,tamanho
561,0.240271,"(Among Us, Apex Legends, ARK: Survival Evolved)",3
562,0.240271,"(Among Us, ARK: Survival Evolved, Assassin's C...",3
563,0.248731,"(Among Us, ARK: Survival Evolved, Battlefield ...",3
564,0.235195,"(Among Us, ARK: Survival Evolved, Call of Duty...",3
565,0.241963,"(Among Us, ARK: Survival Evolved, Counter-Stri...",3
...,...,...,...
6012,0.262267,"(The Witcher 3, Roblox, Valorant)",3
6013,0.247039,"(Rocket League, The Witcher 3, The Sims 4)",3
6014,0.252115,"(Rocket League, The Sims 4, Valorant)",3
6015,0.243655,"(Rocket League, The Witcher 3, Valorant)",3


## Criar regras de associação com base nos itemsets frequentes

In [22]:
# Usando confiança de 40%
regras_associacao = association_rules(itemsets_frequentes, metric='confidence',min_threshold=0.4)
regras_associacao

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
0,(Among Us),(ARK: Survival Evolved),0.514382,0.487310,0.321489,0.625000,1.282552,1.0,0.070825,1.367174,0.453659,0.472637,0.268564,0.642361
1,(ARK: Survival Evolved),(Among Us),0.487310,0.514382,0.321489,0.659722,1.282552,1.0,0.070825,1.427121,0.429703,0.472637,0.299289,0.642361
2,(Apex Legends),(ARK: Survival Evolved),0.519459,0.487310,0.333333,0.641694,1.316809,1.0,0.080196,1.430872,0.500661,0.494975,0.301126,0.662861
3,(ARK: Survival Evolved),(Apex Legends),0.487310,0.519459,0.333333,0.684028,1.316809,1.0,0.080196,1.520834,0.469267,0.494975,0.342466,0.662861
4,(Assassin's Creed Valhalla),(ARK: Survival Evolved),0.490694,0.487310,0.314721,0.641379,1.316164,1.0,0.075601,1.429617,0.471654,0.474490,0.300512,0.643606
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33782,"(The Witcher 3, Valorant)",(The Sims 4),0.331641,0.505922,0.245347,0.739796,1.462272,1.0,0.077562,1.898809,0.472999,0.414286,0.473354,0.612373
33783,"(The Sims 4, Valorant)",(The Witcher 3),0.335025,0.480541,0.245347,0.732323,1.523954,1.0,0.084353,1.940619,0.517031,0.430267,0.484700,0.621443
33784,(The Witcher 3),"(The Sims 4, Valorant)",0.480541,0.335025,0.245347,0.510563,1.523954,1.0,0.084353,1.358653,0.661867,0.430267,0.263977,0.621443
33785,(The Sims 4),"(The Witcher 3, Valorant)",0.505922,0.331641,0.245347,0.484950,1.462272,1.0,0.077562,1.297658,0.639844,0.414286,0.229381,0.612373
